# Image Processing

In [24]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

### Color Channels, the RGB Colorspace

We will see how to load an image, find colors within the image and detect the ball and goal-post

First, we get an image, then separate the color channels and observe how it appears in each channel

In [27]:
img = cv2.imread('view.png')
  
# Displaying the image
cv2.imshow('image', img)

blue = img[:, :, 0]
green = img[:, :, 1]
red = img[:, :, 2]

cv2.imshow('Blue channel', blue)
cv2.imshow('Green channel', green)
cv2.imshow('Red channel', red)

# cv2.imwrite('view_blue.png', blue)
# cv2.imwrite('view_green.png', green)
# cv2.imwrite('view_red.png', red)

True

### Hue, Saturation, Value - HSV Color Space

Since colors are not apparent in the RGB color space, we try the HSV color space

In [41]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)


red_mask = cv2.inRange(hsv, np.array([0,10,10]), np.array([20,255,255]))

# Bitwise-AND mask and original image
red_parts = cv2.bitwise_and(img, img, mask= red_mask)

cv2.imshow('Original Image', img)
cv2.imshow('Red Hue Mask', red_mask)
cv2.imshow('Masked Image', red_parts)

# cv2.imwrite('hue_mask.png', red_mask)
# cv2.imwrite('hue_red.png', red_parts)


Finding the goal post, opponent and the ball

In [52]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)


red_mask = cv2.inRange(hsv, np.array([0,10,10]), np.array([5,255,255]))
orange_mask = cv2.inRange(hsv, np.array([5,10,10]), np.array([20,255,255]))
blue_mask = cv2.inRange(hsv, np.array([100,10,10]), np.array([160,255,255]))

red_parts = cv2.bitwise_and(img, img, mask= red_mask)
orange_parts = cv2.bitwise_and(img, img, mask= orange_mask)
blue_parts = cv2.bitwise_and(img, img, mask= blue_mask)

cv2.imshow('Goal Post', red_parts)
cv2.imshow('Ball', orange_parts)
cv2.imshow('Opponent', blue_parts)

# cv2.imwrite('goal.png', red_parts)
# cv2.imwrite('ball.png', orange_parts)
# cv2.imwrite('opponent.png', blue_parts)

### Image Masks, Denoising and Detecting Connected Regions

For all practical purposes, finding the appropriate masks will suffice.

We use the color masks to detect the ball, opponent and goalpost, find their relative position to our bot and take decisions accordingly.

In [57]:
overlay_image = np.copy(img)

# Red contour
contours = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

for i in contours:
    x,y,w,h = cv2.boundingRect(i)
    cv2.rectangle(overlay_image, (x, y), (x + w, y + h), (255,0,255), 4)
    
# Blue contour
contours = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

for i in contours:
    x,y,w,h = cv2.boundingRect(i)
    cv2.rectangle(overlay_image, (x, y), (x + w, y + h), (255,255,0), 4)
    
# Orange contour
contours = cv2.findContours(orange_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

for i in contours:
    x,y,w,h = cv2.boundingRect(i)
    cv2.rectangle(overlay_image, (x, y), (x + w, y + h), (0,255,255), 4)

    
cv2.imshow('Bounding Box', overlay_image)



We first apply denoising (morphological opening) to remove the rogue pixels from the mask image

In [60]:
kernel = np.ones((5,5),np.uint8)

opening_red = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)
opening_orange = cv2.morphologyEx(orange_mask, cv2.MORPH_OPEN, kernel)
opening_blue = cv2.morphologyEx(blue_mask, cv2.MORPH_OPEN, kernel)

overlay_image = np.copy(img)

# Red contour
contours = cv2.findContours(opening_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

for i in contours:
    x,y,w,h = cv2.boundingRect(i)
    cv2.rectangle(overlay_image, (x, y), (x + w, y + h), (255,0,255), 4)
    
# Blue contour
contours = cv2.findContours(opening_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

for i in contours:
    x,y,w,h = cv2.boundingRect(i)
    cv2.rectangle(overlay_image, (x, y), (x + w, y + h), (255,255,0), 4)
    
# Orange contour
contours = cv2.findContours(opening_orange, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

for i in contours:
    x,y,w,h = cv2.boundingRect(i)
    cv2.rectangle(overlay_image, (x, y), (x + w, y + h), (0,255,255), 4)

    
cv2.imshow('Bounding Box', overlay_image)

# cv2.imwrite('detections.png', overlay_image)


True